In [8]:
# Standard Imports
import numpy as np, pandas as pd, random, json, time, os

# Plotly Imports
import plotly.graph_objects as go
import plotly.express as px

# Dash Imports
import dash
from dash import dcc
from dash import html
#import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate #for multi-option drop
# Additional Imports
# import STUFF

df710 = pd.read_csv("CSV/nsf23300-tab007-010.csv",encoding='cp1252')
df7103 = pd.read_csv("CSV/nsf23300-tab007-010-3.csv",encoding='cp1252')
df107 = df = pd.read_csv("CSV/nsf23300-tab001-007.csv",encoding='cp1252', header=0, index_col=0) # skiprows=[], header=0)
study2 = pd.read_csv("CSV/nsf23300-tab001-007-1.csv",encoding='cp1252', index_col=0)
year = pd.read_csv("CSV/nsf23300-tab001-012-1.csv",encoding='cp1252', header=1, index_col=0, na_values='-') # skiprows=[], header=0)
yearlist= ['1971', '1976', '1981', '1986', '1991', '1996', '2001', '2006', '2011', '2016', '2021']
yeardict= {1971:'1971', 1976:'1976', 1981:'1981', 1986:'1986', 1991:'1991', 1996:'1996', 2001:'2001', 2006:'2006', 2011:'2011', 2016:'2016',2021: '2021'}
# Load CSS sheet for style information
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Define the Web-App Object and set to variable "server"
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# Suppres some output
app.config.suppress_callback_exceptions = True


#################################################
################# Layout ########################
#################################################

app.layout = html.Div([
    #h1 = level 1 header, etc...#
    html.H1(children='Graduate Student Data 1970-2021'),
    html.H2(children='This dashboard helps visualize location and statistics based data of doctorate recipients'),
    
    html.H6("Change below to make a new figure:"),
    
    html.Div([
           "Selection:",
           dcc.Dropdown(['All fields','Agricultural sciences and natural resources','Biological and biomedical sciences','Computer and information sciences','Engineering','Geosciences, atmospheric, and ocean sciences','Health sciences','Mathematics and statistics','Multidisciplinary/ interdisciplinary sciences','Physical sciences','Psychology','Social sciences','Business','Education','Humanities','Visual and performing arts','Other'], 'All fields', id = 'my-input')
    ]),
    html.Div(["Concentration of Graduate Students Across the USA by Field",
            dcc.Graph(id='figure-output')], 
            style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'middle'}),
    html.Br(),

    html.Div([
		"State",
		dcc.Dropdown(['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming'], 'Iowa',id = 'my-input2')
	]),
    
    html.Div([
        "Comparison of Ph.D. Student Fields by State",
        
    html.Div([dcc.Graph(id='figure-output4')], 
            style={'width': '80%', 'display': 'inline-block', 'vertical-align': 'middle'}),
        
        
         html.Div([dcc.Graph(id='figure-output2')], 
            style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'middle'})
    ]),
    
    html.Br(),
    
    html.Div([
        "Field Popularity Selection:",
        dcc.Dropdown(['All doctorate recipientsa', 'Life sciences',
       'Agricultural sciences and natural resources',
       'Biological and biomedical sciences', 'Health sciences',
       'Physical sciences and earth sciences', 'Chemistry',
       'Geosciences, atmospheric sciences, and ocean sciences',
       'Physics and astronomy', 'Mathematics and computer sciences',
       'Computer and information sciences', 'Mathematics and statistics',
       'Psychology and social sciences', 'Psychology', 'Anthropology',
       'Economics', 'Political science and government', 'Sociology',
       'Other social sciences', 'Engineering',
       'Aerospace, aeronautical, and astronautical engineering',
       'Bioengineering and biomedical engineering', 'Chemical engineering',
       'Civil engineering',
       'Electrical, electronics, and communications engineering',
       'Industrial and manufacturing engineering',
       'Materials science engineering', 'Mechanical engineering',
       'Other engineering', 'Education', 'Education administration',
       'Education research', 'Teacher education', 'Teaching fields',
       'Other education', 'Humanities and arts',
       'Foreign languages and literature', 'History', 'Letters',
       'Other humanities and arts', 'Other',
       'Business management and administration', 'Communication',
       'Non-science and engineering fields nec'
       ], ['Chemistry','History','Letters'],id ='my-input3', multi=True )
    ]),
        

        html.Div(["Graduate Study Popularity Comparison",dcc.Graph(id='figure-output3')], 
            style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'middle'}),


    html.Br(),
    
        
    #html.Div([
     #   "Selection Year:",
      #  dcc.Slider(1971, 1976, 1981, 1986, 1991, 1996, 2001, 2006, 2011, 2016, 2021
       #            , value =1971,
        #       id='my-input5')]),
    html.Div([
        "Sample Year",
        dcc.Slider(1971,2021,5, 
                   #marks={a: {'label':yearlist[a]} for a in range(0,11)}, 
                   marks=yeardict, value=1971, 
                   
                    id='my-input5')
        ]),
        
    html.Div([
        
        html.Div(["Years to Ph.D. Completion by Field",
            dcc.Graph(id='figure-output5')],
            style = {'width': '98%', 'display': 'inline-block', 'vertical-align': 'middle'})])
    

]) 
    
    

                                                                                           
                                                                                    
    
#   def limit_drop_options(symbols):
#    """Limit dropdown to at most two actives selections"""
#    if len(symbols) >= 5:

 #     children = html.P("You have entered the limit", id='input-warning')
 #     return [
 #         option
 #         for option in options
 ##         if option["value"] in symbols
  #        children
          
   #   ]
   # else:
    #    return options



#####################
# Plot one          #
#####################
@app.callback(
    Output('figure-output', 'figure'),
    Input('my-input', 'value'))    
def make_plot(pltcolor):
    hi = df710[pltcolor].max()
    low = df710[pltcolor].min()
    fig = px.choropleth(df710,
                    locations = 'Abbrev',
                    locationmode = 'USA-states',
                    scope = 'usa',
                    color = pltcolor,
                    hover_name = 'State or location',
                    #hover_data = ['Male','Female'],
                    range_color = [low,hi],
                    color_continuous_scale = 'curl')
    return fig  
    
#####################
# Plot two          #
#####################
@app.callback(
    Output('figure-output2', 'figure'),
    Input('my-input2', 'value'))    
def make_plot(state):
    fig = px.pie(df7103, 'State or location', state, color_discrete_sequence=px.colors.diverging.curl)
    return fig    


#####################
# Plot three          #
#####################
@app.callback(
        Output('figure-output3', 'figure'),
        Input('my-input3', 'value')
   
    )
def plotthree(z):
    
    n=0
    for i in z:
        n+=1
        if n==1:
            firstbar = study2[i].tolist()#[study2[i][j] for j in range(11)] #MUST BE DYNAMIC, USED THE Number
        elif n==2:
        
            secondbar = study2[i].tolist()#[study2[i][j] for j in range(11)]  #MUST BE DYNAMIC
        elif n==3:
            thirdbar = study2[i].tolist()#[study2[i][j] for j in range(11)]   # MUST be DYNAMIC
        else:
            break
    branches = ['2011', '2012 ', '2013', '2014','2015','2016','2017','2018','2019','2020','2021']
   
    
    #TRACES must be processed as a list so it deals with all elements in the list rather than give a number
    trace1 = go.Bar(
        x = branches,
        
        y = firstbar,#[firstbar[i] for i in range(11)],
        
        name = z[0], marker_color='#6f175b'  ## MUST be DYNAMIC, but dependent on the input in a slightly different way perhaps? (comes from column header..?)
       
    )
    
    trace2 = go.Bar(
        x = branches,

        y = secondbar,#[secondbar[i] for i in range(11)],

        name = z[1], marker_color='#e8baa8'

    )
    trace3 = go.Bar(
        x = branches,

        y = thirdbar,#[firstbar[i] for i in range(11)],

        name = z[2], marker_color='#136e72'

    )
    data = [trace1, trace2, trace3]
    layout = go.Layout(barmode = 'group')
    fig = go.Figure(data = data, layout = layout)
    return fig
   
#####################
# Plot four          #
#####################
@app.callback(
    Output('figure-output4', 'figure'),
    Input('my-input2', 'value'))    
def make_plot(state):
    fig = px.bar(df7103, 'State or location', state, color = 'State or location',color_discrete_sequence=px.colors.diverging.curl)#, color_discrete_sequence=px.colors.diverging.Armyrose)
    fig.update_layout(xaxis = {"categoryorder":"total descending"},)
    #fig.update_layout(title="Model Output")
    
    return fig  



#####################
#Plot Five           ##
#####################
@app.callback(
    Output('figure-output5', 'figure'),
    Input('my-input5', 'value'))


def make_finalplot(yr):
    #print(yr)
    yr=str(yr)
    #year = pd.read_csv("CSV/nsf23300-tab001-012-1.csv",encoding='cp1252', header=1, index_col=0, na_values='-') 
    #'Median years to research doctorate, by historical major field of doctorate: Selected years, 1971–2021'
    for i in [1971,1976,1981,1986,1991,1996,2001,2006,2011,2016,2021]:
        for a in year[str(i)]:
            years=year.astype({str(i):'float'})
    Field=years.index
   # counts=[year[str(yr)][n] for n in range(len(year[str(yr)]))]
    #fig = px.bar(years.index, counts, color = years.index, color_discrete_sequence=px.colors.diverging.Armyrose, height=700)#, color_discrete_sequence=px.colors.diverging.Armyrose)
    ##fig.update_layout(xaxis = {"categoryorder":"total ascending"}, showlegend=False)
    #fig.update_yaxes(dtick=2, title='Median Years to Completion')
    fig = px.bar(years.index,Field,years[str(yr)], color = years.index, color_discrete_sequence=px.colors.diverging.curl, height=700)#, color_discrete_sequence=px.colors.diverging.Armyrose)
    fig.update_layout(xaxis = {"categoryorder":"total ascending"}, showlegend=False)
    fig.update_yaxes(dtick=2, title='Median Years to Completion')
    
    return fig








# -------------------------- MAIN ---------------------------- #


# This is the code that gets run when we call this file from the terminal
# The port number can be changed to fit your particular needs
if __name__ == '__main__':
    app.run_server(host='localhost', port=8080, debug=True, use_reloader=False)
#v5 11:24

Dash is running on http://localhost:8068/

 * Serving Flask app '__main__'
 * Debug mode: on
